<a href="https://colab.research.google.com/github/Anshuman-37/Summer_Dissertation/blob/main/First_Model_Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Import Files

In [2]:
import os
import re
import nibabel as nib
import numpy as np
import torch
print(torch.__version__)
torch.cuda.is_available()

import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, LazyConv3d , MaxPool3d, Module, Softmax, BatchNorm3d, Dropout
from torch.optim import Adam, SGD

1.11.0+cu113


##### DATA LOADING

In [ ]:
!unzip /content/Raw\ Files.zip -d /content/
!unzip /content/Baseline.zip -d /content/

###### MRI Data

In [3]:
## Go in to the directories
## Extracting only important data using Regex
path = '/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/'
regex = re.compile(r'__MPR.*') ; fileRegex1 = re.compile(r'(WIP)*T13D'); mri_data_path = []; 

# Iterating over Directories 
for subdir, dirs, files in os.walk(path):
    for file in files:
        # Storing the path of Data
        if regex.search(file) == None:
            if fileRegex1.search(file): mri_data_path.append(os.path.join(subdir, file));

# Printing the path of Data
for i in mri_data_path: print(i);

mri_images = []
# Loading data in the image vector
for i in mri_data_path: mri_images.append(nib.load(i)); 

# Printing the shape of images stored and its data type
for i in mri_images: print('Image shape ->',i.shape,end = '\t'); print('Image Data Type ->',i.get_data_dtype());

# To create the animation of the MRI images
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation


/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1277547 dicom/__WIP_MPRAGE_T13D_SENSE_20150430160250_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1327832 dicom/__WIP_MPRAGE_T13D_SENSE_20160222133236_301.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1335478 dicom/__WIP_MPRAGE_T13D_SENSE_20160407112645_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1289317 dicom/__WIP_MPRAGE_T13D_SENSE_20150710135121_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1320961 dicom/__WIP_MPRAGE_T13D_SENSE_20160115152740_301.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1291194 dicom/__WIP_MPRAGE_T13D_SENSE_20150724132211_401.nii
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288, 180)	Image Data Type -> int16
Image shape -> (288, 288

##### ASL Data


In [4]:
# Getting the path for ASL files
asl_path = '/content/Baseline/'
asl_data_path = []
asl_regex = re.compile(r'oxasl_dist.*') 
# Iterating over directories
for subdir, dirs, files in os.walk(asl_path):
    # Selecting the ASL file
    for file in files:
        if file == 'perfusion_calib.img':
            if asl_regex.search(subdir): 
                asl_data_path.append(os.path.join(subdir, file));

# Printing the path of ASL images
for i in asl_data_path: print(i);

asl_images = []

# Regex for the asl image

for i in asl_data_path: asl_images.append(nib.load(i)); #print(i); #asl_images.append(nib.load(i)); 
# # Loading data in the image vector
# for i in asl_data_path: asl_images.append(nib.load(i)); 

# # Printing the shape of images stored and its data type
for i in asl_images: print('Image shape ->',i.shape,end = '\t'); print('Image Data Type ->',i.get_data_dtype());

/content/Baseline/1277547 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.img
/content/Baseline/1327832 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.img
/content/Baseline/1289317 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.img
/content/Baseline/1291194 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.img
Image shape -> (80, 80, 13)	Image Data Type -> float32
Image shape -> (80, 80, 13)	Image Data Type -> float32
Image shape -> (80, 80, 13)	Image Data Type -> float32
Image shape -> (80, 80, 13)	Image Data Type -> float32


In [6]:
# img = asl_images[0]
# a = np.array(img.dataobj)[:,:,:]
# for i in range(0,13):
#     plt.imshow(a[:,:,i],cmap = 'gray')
#     plt.show()

## Getting X and Y


In [5]:

print('MRI DATA DICT \t \n','-'*50);
### Data hashmaps - 
### Mri Data  ## Idea - Patient number(key) - Path Stored(value)
mri_map = {};

## Iterating over the mri data's path
for i in mri_data_path:
    ## Value is our path i.e. stored and key is the patient number
    value = i; key = i.split('/')[4].split(' ')[0]; mri_map[key] = value;

# Printing the mri_data_dict 
for k,v in mri_map.items(): print(k,'->',v);

######
print('\nASL DATA DICT - \t \n','-'*50);
######
### ASL Data ## Idea - Patient number(key) - Path Stored(value)
asl_map = {}

## Iterating over the mri data's path
for i in asl_data_path:
    ## Value is our path i.e. stored and key is the patient number
    value = i; key = i.split('/')[3].split(' ')[0]; asl_map[key] = value;

# Printing the asl_data_dict 
for k,v in asl_map.items(): print(k,'->',v);

MRI DATA DICT 	 
 --------------------------------------------------
1277547 -> /content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1277547 dicom/__WIP_MPRAGE_T13D_SENSE_20150430160250_401.nii
1327832 -> /content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1327832 dicom/__WIP_MPRAGE_T13D_SENSE_20160222133236_301.nii
1335478 -> /content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1335478 dicom/__WIP_MPRAGE_T13D_SENSE_20160407112645_401.nii
1289317 -> /content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1289317 dicom/__WIP_MPRAGE_T13D_SENSE_20150710135121_401.nii
1320961 -> /content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1320961 dicom/__WIP_MPRAGE_T13D_SENSE_20160115152740_301.nii
1291194 -> /content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1291194 dicom/__WIP_MPRAGE_T13D_SENSE_20150724132211_401.nii

ASL DATA DICT - 	 
 --------------------------------------------------
1277547 -> /content/Baseline/1277547 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.i

In [6]:
## Lists to get train x and train y differently
train_x_path = [] ; train_y_path = []; 

### Traversing over the map and getting the values for which asl data is present
for k,v in mri_map.items():
    if k in asl_map: train_x_path.append(v); train_y_path.append(asl_map[k]);

## Printing train_x and train_y to get path stored
for i in train_x_path: print(i); 
for i in train_y_path: print(i);

/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1277547 dicom/__WIP_MPRAGE_T13D_SENSE_20150430160250_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1327832 dicom/__WIP_MPRAGE_T13D_SENSE_20160222133236_301.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1289317 dicom/__WIP_MPRAGE_T13D_SENSE_20150710135121_401.nii
/content/Raw Files/Raw_nii_files_for_MC_pCASL_T1_B0_M0/1291194 dicom/__WIP_MPRAGE_T13D_SENSE_20150724132211_401.nii
/content/Baseline/1277547 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.img
/content/Baseline/1327832 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.img
/content/Baseline/1289317 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.img
/content/Baseline/1291194 dicom/analysis/oxasl_distcorr/native_space/perfusion_calib.img


In [17]:
x = [] ; y = [];

for k,v in mri_map.items():
    if k in asl_map:
        # Loading the MRI image from the path in the train x path 
        mri_img = nib.load(v); 
        # Making it a numpy array
        mri_vec = np.array(mri_img.dataobj)[np.newaxis,:,:,:] # Channels x Length X Breadth X Slices of Brain
        # Min max Normalizing the image 
        mri_vec = (mri_vec - mri_vec.min()) / (mri_vec.max() - mri_vec.min())
        # Appending the MRI image to X 
        x.append(torch.as_tensor(mri_vec,dtype=torch.float32)); 
        ## Finding the same patient with ASL data 
        asl_img = nib.load(asl_map[k]); asl_vec = np.array(asl_img.dataobj)[np.newaxis,:,:,:] ; asl_vec = (asl_vec - asl_vec.min()) / (asl_vec.max() - asl_vec.min());
        ## Appending the image to y
        y.append(torch.as_tensor(asl_vec,dtype=torch.float32))
    

In [18]:

# MRI Data shape
for i in x : print(i.shape , i.dtype)
# ASL Data shape
for i in y : print(i.shape , i.dtype)


torch.Size([1, 288, 288, 180]) torch.float32
torch.Size([1, 288, 288, 180]) torch.float32
torch.Size([1, 288, 288, 180]) torch.float32
torch.Size([1, 288, 288, 180]) torch.float32
torch.Size([1, 80, 80, 13]) torch.float32
torch.Size([1, 80, 80, 13]) torch.float32
torch.Size([1, 80, 80, 13]) torch.float32
torch.Size([1, 80, 80, 13]) torch.float32


#### Model

In [19]:
t = torch.as_tensor(np.array(nib.load(mri_map['1277547']).dataobj))
print(t.shape, t.dtype)
print(t.sum(), t.prod(), t.mean(), t.std())

torch.Size([288, 288, 180]) torch.float64
tensor(4.9879e+11, dtype=torch.float64) tensor(0., dtype=torch.float64) tensor(33408.9108, dtype=torch.float64) tensor(53164.6250, dtype=torch.float64)


In [20]:
tensor_mri = x[0] ; print(tensor_mri.sum(), tensor_mri.prod(), tensor_mri.mean(), tensor_mri.std());
tensor_asl = y[0] ; print(tensor_asl.sum(), tensor_asl.prod(), tensor_asl.mean(), tensor_asl.std()); 

tensor(1256233.) tensor(0.) tensor(0.0841) tensor(0.1339)
tensor(5888.7588) tensor(0.) tensor(0.0708) tensor(0.1548)


In [21]:
## Go with Lazy 3D convs LAZYCONV3Dw

## The loss function to get the result
class Normal_3D_cnn(Module):   
    
    def __init__(self):
        super(Normal_3D_cnn, self).__init__()
        
        self.cnn_layers = Sequential(
            # Defining a 3D convolution layer
            LazyConv3d(1, kernel_size= 3, stride=1, padding=1),
            BatchNorm3d(1),
            ReLU(inplace=True),
            MaxPool3d(kernel_size=2, stride=2),
            # # Defining another 3D convolution layer
            # LazyConv3d(1,kernel_size=3, stride=1, padding=1),
            # BatchNorm3d(1),
            # ReLU(inplace=True),
            # MaxPool3d(kernel_size=2, stride=2),
        )
    def forward(self, x):
        x = self.cnn_layers(x)
        return x

model = Normal_3D_cnn()
print(model)
optimizer = Adam(model.parameters(), lr=0.07)

def train():
    model.train()
    optimizer.zero_grad()
    y_p = model(x[0])
    loss = mse(y_t,y_p)
    return loss, y_p

loss,y_p = train()
print(loss,y_p.shape)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Normal_3D_cnn(
  (cnn_layers): Sequential(
    (0): LazyConv3d(0, 1, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)


ValueError: ignored

###### LOSS

In [22]:
def mse(y_t , y_p):
    ## To calculate the mean square error
    print(y_t.shape); print(y_p.shape); 
    ## Normal Mean Square Error - reduce( x+y , (y_t - y_p)^2)
    loss = torch.sum(torch.square(torch.flatten(y_t) - torch.flatten(y_p)))
    return loss
print(mse(y[1],y[2]))

torch.Size([1, 80, 80, 13])
torch.Size([1, 80, 80, 13])
tensor(991.0825)


##### Training
